In [ ]:
import os
import itertools
import json
import ast

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from joblib import dump

RANDOM_STATE = 42
DATA_IN = "New_York_Airbnb_Enhanced_Unclean.csv"
OUT_FINAL = "Airbnb_PowerBI_Final_Dataset.csv"
OUT_GRID = "price_suggestion_grid.csv"

# -------------------------
# Helper preprocessing
# -------------------------
def count_amenities(x):
    if pd.isna(x):
        return 0
    # amenities are often stored like: "{'Wifi','TV',...}" or '["Wifi","TV"]'
    try:
        # handle both python-list-like and quoted lists
        val = ast.literal_eval(x)
        if isinstance(val, (list, set, tuple)):
            return len(val)
        if isinstance(val, dict):
            return len(val)
    except Exception:
        # fallback: count commas inside string (cheap heuristic)
        try:
            s = str(x)
            return max(0, s.count(",") + 1)
        except:
            return 0
    return 0

def load_and_clean(path):
    df = pd.read_csv(path)

    # Keep a copy of original few columns if desired
    # Remove rows with missing price or price <= 0
    df = df[df['price'].notna()]
    df = df[df['price'] > 0]

    # Remove extreme outliers in price (cap at 99th percentile and/or absolute threshold)
    pct99 = df['price'].quantile(0.99)
    df = df[df['price'] <= min(pct99, 500)]  # limit to 500 or 99th percentile

    # Amenities -> count
    if 'amenities' in df.columns:
        df['amenities_count'] = df['amenities'].apply(count_amenities)
        df.drop(columns=['amenities'], inplace=True)

    # Fill missing reviews_per_month -> 0
    if 'reviews_per_month' in df.columns:
        df['reviews_per_month'].fillna(0, inplace=True)

    # Basic fill for other common fields if missing
    fill_zero_cols = ['number_of_reviews', 'minimum_nights', 'calculated_host_listings_count', 'availability_365']
    for c in fill_zero_cols:
        if c in df.columns:
            df[c].fillna(0, inplace=True)

    # Create log transforms for skewed columns
    df['price_log'] = np.log1p(df['price'])
    df['reviews_log'] = np.log1p(df['number_of_reviews']) if 'number_of_reviews' in df.columns else 0
    df['nights_log'] = np.log1p(df['minimum_nights']) if 'minimum_nights' in df.columns else 0

    # Convert boolean columns to int if any
    bool_cols = df.select_dtypes(include=['bool']).columns
    if len(bool_cols) > 0:
        df[bool_cols] = df[bool_cols].astype(int)

    # Keep features we will likely use
    # We'll choose a pragmatic set: neighbourhood_group, room_type, latitude, longitude,
    # calculated_host_listings_count, availability_365, amenities_count, reviews_log, nights_log
    selected_cols = [
        'neighbourhood_group', 'room_type', 'latitude', 'longitude',
        'calculated_host_listings_count', 'availability_365',
        'amenities_count', 'reviews_log', 'nights_log', 'price_log'
    ]
    # Ensure selected cols exist; if not, adapt to what's available
    present_cols = [c for c in selected_cols if c in df.columns]
    df = df.copy()  # avoid SettingWithCopyWarning

    return df, present_cols

# -------------------------
# Prepare features for ML (one-hot encode categorical + scaling)
# -------------------------
def prepare_features(df, feature_cols, fit_meta=None):
    """
    feature_cols: list of columns we will use (must include price_log not here)
    fit_meta: if provided, must contain 'columns' list used for reindexing and 'scaler'
    returns X (DataFrame), meta dict
    """
    df_local = df.copy()
    categorical = [c for c in ['neighbourhood_group', 'room_type'] if c in df_local.columns and c in feature_cols]
    numeric = [c for c in feature_cols if c not in categorical]

    # One-hot
    if categorical:
        df_o = pd.get_dummies(df_local[categorical], drop_first=True)
        df_local = pd.concat([df_local[numeric], df_o], axis=1)
    else:
        df_local = df_local[numeric]

    # Order columns consistently
    if fit_meta is None:
        cols = list(df_local.columns)
    else:
        # Reindex to match training columns, add missing cols as zeros
        cols = fit_meta['columns']
        df_local = df_local.reindex(columns=cols, fill_value=0)

    # Scaling: we can scale for models that benefit - XGBoost doesn't require scaling,
    # but scaling is harmless if we save the scaler and apply consistently. We will NOT scale by default
    meta = {}
    meta['columns'] = list(df_local.columns)
    return df_local, meta

# -------------------------
# Main
# -------------------------
def main():
    print("Loading and cleaning data...")
    df, present_cols = load_and_clean(DATA_IN)

    # select features for training (exclude the target price_log)
    # We prefer these features if present:
    want = ['neighbourhood_group', 'room_type', 'latitude', 'longitude',
            'calculated_host_listings_count', 'availability_365',
            'amenities_count', 'reviews_log', 'nights_log']
    feature_cols = [c for c in want if c in df.columns]

    print("Features used:", feature_cols)

    # Prepare features (fit)
    X_df, meta = prepare_features(df, feature_cols, fit_meta=None)
    y = df['price_log']

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=RANDOM_STATE)
    print(f"Training rows: {X_train.shape[0]}, Test rows: {X_test.shape[0]}")

    # Train XGBoost
    print("Training XGBoost...")
    xgb = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbosity=0
    )
    xgb.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)
    


    # Create final export DataFrame: add predicted price and price difference
    print("Creating final export CSV with predictions...")
    # Ensure X_df columns used in training (meta['columns'])
    X_for_pred = X_df[meta['columns']]

    preds_log = xgb.predict(X_for_pred)
    preds_price = np.expm1(preds_log)

    df_export = df.copy()
    # revert transformed fields to original-ish values for readability
    df_export['price'] = np.expm1(df_export['price_log'])
    df_export['reviews'] = np.expm1(df_export['reviews_log'])
    df_export['nights'] = np.expm1(df_export['nights_log'])

    df_export['Predicted_Price'] = preds_price
    df_export['Price_Suggestion_Diff'] = df_export['Predicted_Price'] - df_export['price']

    # Save CSV
    df_export.to_csv(OUT_FINAL, index=False)
    print(f"Wrote final dataset to {OUT_FINAL} ({df_export.shape[0]} rows)")

    # -------------------------
    # Create prediction grid for Power BI slider
    # -------------------------
    print("Creating prediction grid for Power BI (slider)...")
    # Choose variables to expose in slider: room_type, neighbourhood_group, minimum_nights, amenities_count
    # Use values from the cleaned df to limit cardinality
    room_types = df['room_type'].dropna().unique().tolist() if 'room_type' in df.columns else ['home/apt']
    neighbourhoods = df['neighbourhood_group'].dropna().unique().tolist() if 'neighbourhood_group' in df.columns else ['Manhattan']

    # reasonable ranges:
    min_nights_vals = list(range(1, 31))  # 1..30
    amenities_vals = list(range(0, max(31, int(df['amenities_count'].max() + 1))))  # 0..max

    # To avoid an enormous grid, limit combinations: we will create full cross-product of these
    combos = list(itertools.product(room_types, neighbourhoods, min_nights_vals, amenities_vals))
    print(f"Grid will have {len(combos):,} rows (room_type * neighbourhood * min_nights * amenities_count)")

    grid_df = pd.DataFrame(combos, columns=['room_type', 'neighbourhood_group', 'minimum_nights', 'amenities_count'])

    # Add other numeric columns with reasonable default values / medians
    # Use median values from training df for features not in the grid
    medians = {}
    for c in ['latitude', 'longitude', 'calculated_host_listings_count', 'availability_365', 'reviews_log', 'nights_log']:
        medians[c] = df[c].median() if c in df.columns else 0

    # Add columns used by model: latitude/longitude etc.
    for c, v in medians.items():
        grid_df[c] = v

    # For minimum_nights and reviews fields we want to ensure their transformed versions are aligned
    # We included 'nights_log' as a feature for training. The grid includes minimum_nights; we must create nights_log
    grid_df['nights_log'] = np.log1p(grid_df['minimum_nights'])
    # reviews_log use median (already set)

    # Now prepare the grid features the same way we prepared training data:
    # We must ensure one-hot columns match training columns (meta['columns'])
    # First, create a temporary df in same structure as df used in prepare_features
    # We need to include the categorical cols and numeric cols
    # Build a df that has all the feature columns before one-hot
    pre_grid = pd.DataFrame()
    # Copy categorical and numeric raw columns used in the original feature_cols
    if 'neighbourhood_group' in feature_cols:
        pre_grid['neighbourhood_group'] = grid_df['neighbourhood_group']
    if 'room_type' in feature_cols:
        pre_grid['room_type'] = grid_df['room_type']
    # numeric features
    for c in ['latitude', 'longitude', 'calculated_host_listings_count', 'availability_365', 'amenities_count', 'reviews_log', 'nights_log']:
        if c in feature_cols:
            pre_grid[c] = grid_df[c]

    # One-hot encode grid in the same way as training df, and then reindex to meta['columns']
    grid_o = pd.get_dummies(pre_grid[[c for c in ['neighbourhood_group', 'room_type'] if c in pre_grid.columns]], drop_first=True) if any([c in pre_grid.columns for c in ['neighbourhood_group','room_type']]) else pd.DataFrame(index=pre_grid.index)
    numeric_grid = pre_grid[[c for c in pre_grid.columns if c not in ['neighbourhood_group', 'room_type']]]
    grid_features = pd.concat([numeric_grid.reset_index(drop=True), grid_o.reset_index(drop=True)], axis=1)
    # Reindex to training columns
    training_cols = meta['columns']
    grid_features = grid_features.reindex(columns=training_cols, fill_value=0)

    # Predict on grid
    grid_preds_log = xgb.predict(grid_features)
    grid_preds_price = np.expm1(grid_preds_log)
    grid_df['predicted_price'] = grid_preds_price

    # Save grid
    grid_df.to_csv(OUT_GRID, index=False)
    print(f"Wrote prediction grid to {OUT_GRID} ({grid_df.shape[0]} rows)")

    print("All done. Files created:")
    print(" -", OUT_FINAL)
    print(" -", OUT_GRID)

if __name__ == "__main__":
    main()


Loading and cleaning data...
Features used: ['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'calculated_host_listings_count', 'availability_365', 'amenities_count', 'reviews_log', 'nights_log']
Training rows: 3922, Test rows: 981
Training XGBoost...


C:\Users\sh_m_\AppData\Local\Temp\ipykernel_21332\2468008277.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews_per_month'].fillna(0, inplace=True)
C:\Users\sh_m_\AppData\Local\Temp\ipykernel_21332\2468008277.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

Saved model to xgb_model.joblib and meta to preprocess_meta.joblib
Creating final export CSV with predictions...
Wrote final dataset to Airbnb_PowerBI_Final_Dataset.csv (4903 rows)
Creating prediction grid for Power BI (slider)...
Grid will have 13,950 rows (room_type * neighbourhood * min_nights * amenities_count)
Wrote prediction grid to price_suggestion_grid.csv (13950 rows)
All done. Files created:
 - Airbnb_PowerBI_Final_Dataset.csv
 - price_suggestion_grid.csv
 - xgb_model.joblib
 - preprocess_meta.joblib
